In [ ]:
import numpy as np
import pandas as pd
import sqlite3
from tqdm import tqdm

class Preprocessing:


  def import_data(self):
    conn = sqlite3.connect('office4.db')
    cur_1 = conn.cursor()

    conn = sqlite3.connect('h355-2015.sqlite3')
    cur_2 = conn.cursor()
    return cur_1, cur_2

  def loading_feature_office_1(self, row, cur_1):

    cur_1.execute("SELECT * FROM "+ row)
    rows = cur_1.fetchall()
    data = pd.DataFrame(data = rows)
    data.columns = ['time', 'value', 'date']
    data = data.sort_values('date')
    return data


  def loading_class_office_1(self, cur_1):
    cur_1.execute("SELECT * FROM 'labels'")
    rows = cur_1.fetchall()
    data = pd.DataFrame(data = rows)
    data.columns = ['date', 'RMS', 'number_occupants', 'time', 'value']
    data = data.sort_values('date')
    return data


  def create_feature_office_1(self, x):
    new_time = int(list(time_value.keys())[0]) + (x*1000)
    #print("new_time", new_time)
    #print("len data:",len(data))
    for i in tqdm(range(len(data))):

      if (new_time >= int(list(time_value.keys())[i])) and (new_time < int(list(time_value.keys())[i+1])):
        res = float(list(time_value.values())[i])
        #print("res", res)
        break
    return res

  def data_processing_office_1(self):
    global time_value
    global data

    columns = ['co2','RMS', 'power1', 'power2', 'power3', 'power4', 'power5',
       'power6', 'OCCUPANCY', 'corridor', 'Temperature', 'OUTDOOR_TEMPERATURE',
       'Door_contact', 'window', 'labels']

    Final_Data = pd.DataFrame(data = np.arange(0,3600*24*10,600), columns = ['time'])

    cur_1, cur_2 = self.import_data()
    for i in columns:
      if i == 'labels':
        data = self.loading_class_office_1(cur_1)
        time_value = {data.iloc[j,3]:data.iloc[j,2] for j in range(len(data))}
      else:
        data = self.loading_feature_office_1(i, cur_1)
        time_value = {data.iloc[j,0]:data.iloc[j,1] for j in range(len(data))}

      create_feature = self.create_feature_office_1
      Final_Data[i] = Final_Data['time'].apply(create_feature)

    Final_Data.labels = Final_Data.labels.round()
    return Final_Data

  def loading_feature_office_2(self, cur_2):

    cur_2.execute("SELECT * From 'data'")
    rows = cur_2.fetchall()
    data1 = pd.DataFrame(data = rows)
    data2 = pd.read_csv("rms.csv", header = None)
    data = pd.concat([data1, data2])
    data.columns = ['variableidref','time', 'value', 'date']
    data['date'] = pd.to_datetime(data['date'], format='%d/%m/%Y %H:%M:%S')
    data = data.sort_values('time')

    data = data[pd.to_datetime(data.date) >= pd.to_datetime('14/03/2016 04:53:16')]
    data = data[pd.to_datetime(data.date) <= pd.to_datetime('21/03/2016 15:00:16')]

    return data

  def loading_class_office_2(self):
    data = pd.read_csv("labels_H355.csv")
    data.columns = ['time', 'value', 'date']
    return data

  def create_feature_office_2(self,x):
    new_time = int(list(time_value.keys())[0]) + (x*1000)
    for i in tqdm(range(len(data))):
      try:

        if (new_time >= int(list(time_value.keys())[i])) and (new_time < int(list(time_value.keys())[i+1])):
          res = list(time_value.values())[i]
          break
      except: res = list(time_value.values())[i]
    return res

  def create_class_office_2(self,x):
    new_time = pd.to_datetime('14/03/2016 04:53:16') + pd.to_timedelta(x, unit='s')

    for i in range(len(data)):
      try:
        res = 0
        if (new_time >= pd.to_datetime(list(time_value.keys())[i])) and (new_time < pd.to_datetime(list(time_value.keys())[i+1])):
          res = list(time_value.values())[i]
          break
      except : res = list(time_value.values())[i]
    return res


  def data_processing_office_2(self):
    global time_value
    global data

    columns = {'illuminance':1,'rain':2, 'windspeed':4,'day/night':0, 'RMS':30, 'power2':11, 'power3':14, 'power4': 23,'co2':15, 'Occupancy':19,'power1':5, 'Temperature': 17, 'Temperature1': 10, 'Temperature12': 7, 'door1': 13, 'window':12}
    #'RMS':30,'power1':5,  'power2':11, 'power3':14, 'power4': 23, 'Occupancy':19, 'window':12
    #'power1':5,'co2':15, 'RMS':30, 'power2':11, 'power3':14, 'power4': 23, 'Occupancy':19, 'Temperature': 17, 'Temperature1': 10, 'Temperature12': 7, 'door1': 13, 'window':12
    Final_Data = pd.DataFrame(data = np.arange(0,3600*24*7 + 10*3600 ,600), columns = ['time'])

    cur_1, cur_2 = self.import_data()
    new_data = self.loading_feature_office_2(cur_2)
    for j in columns:
      data = new_data[new_data.variableidref == columns[j]]
      time_value = {data.iloc[i,1]:data.iloc[i,2] for i in range(len(data))}
      print("time_value: ", time_value)
      create_feature = self.create_feature_office_2
      Final_Data[j] = Final_Data.time.apply(create_feature)

    data = self.loading_class_office_2()
    time_value = {data.iloc[i,2]:data.iloc[i,1] for i in range(len(data))}
    transform_label = self.create_class_office_2
    Final_Data['labels'] = Final_Data.time.apply(transform_label)

    return Final_Data




#script

preprocessing = Preprocessing()
Final_Data_S1 = preprocessing.data_processing_office_1()
Final_Data_S1.to_csv("Final_Data_S1.csv")

Final_Data_S2 = preprocessing.data_processing_office_2()
Final_Data_S2.to_csv("Final_Data_S2_2.csv")